In [8]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences

max_features = 2000  # Top 10,000 words
maxlen = 200  # First 200 words of each review

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

# Add start and end tokens
start_token = max_features + 1
end_token = max_features + 2

X_train = [ [start_token] + seq + [end_token] for seq in X_train ]
X_test = [ [start_token] + seq + [end_token] for seq in X_test ]

# Update max_features to include start and end tokens
max_features += 2

# Pad sequences
X_train = pad_sequences(X_train, maxlen=maxlen, padding='post')
X_test = pad_sequences(X_test, maxlen=maxlen, padding='post')


# Pad sequences
X_train = pad_sequences(X_train, maxlen=maxlen, padding='post')
X_test = pad_sequences(X_test, maxlen=maxlen, padding='post')

In [15]:
import os
os.environ['TF_GPU_ALLOCATOR']='cuda_malloc_async'
import keras
from keras import layers
# Parameters
batch_size = 512       # Batch size for training

# Decoder input sequences: shifted right by one position
decoder_input_data = np.zeros_like(X_train)
decoder_input_data[:, :-1] = X_train[:, 1:]

# Decoder target sequences: the original sequences
decoder_target_data = np.zeros_like(X_train)
decoder_target_data = X_train


from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model

embedding_dim = 128
latent_dim = 256  # Adjust as needed

# Encoder
encoder_inputs = Input(shape=(maxlen,), name='encoder_inputs')
encoder_embedding = Embedding(input_dim=max_features, output_dim=embedding_dim, mask_zero=True, name='encoder_embedding')(encoder_inputs)
_, state_h, state_c = LSTM(latent_dim, return_state=True, name='encoder_lstm')(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(maxlen,), name='decoder_inputs')
# Define the embedding layer separately
decoder_embedding_layer = Embedding(
    input_dim=max_features,
    output_dim=embedding_dim,
    mask_zero=True,
    name='decoder_embedding'
)
decoder_embedding = decoder_embedding_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_lstm')
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(max_features, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')





In [16]:
decoder_target_data = np.expand_dims(decoder_target_data, -1)

history = model.fit(
    [X_train, decoder_input_data],
    decoder_target_data,
    batch_size=64,
    epochs=10,
    validation_split=0.2
)



Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 13s 36ms/step - loss: nan - val_loss: nan
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - loss: nan - val_loss: nan
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - loss: nan - val_loss: nan
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - loss: nan - val_loss: nan
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - loss: nan - val_loss: nan
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - loss: nan - val_loss: nan
Epoch 7/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - loss: nan - val_loss: nan
Epoch 8/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - loss: nan - val_loss: nan
Epoch 9/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - loss: nan - val_loss: nan
Epoch 10/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - loss: nan - val_loss: nan


In [11]:
encoder_model = Model(encoder_inputs, encoder_states)




## 

In [17]:
from keras.layers import Input

# Decoder setup for inference

# Input tensors for hidden and cell states
decoder_state_input_h = Input(shape=(latent_dim,), name='input_h')
decoder_state_input_c = Input(shape=(latent_dim,), name='input_c')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# New decoder input for inference (single time step)
decoder_inputs_single = Input(shape=(1,), name='decoder_input_inference')

# Get embeddings (reuse embedding layer)
decoder_embedding2 = decoder_embedding_layer(decoder_inputs_single)

# Predict next word and states (reuse LSTM layer)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(
    decoder_embedding2, initial_state=decoder_states_inputs)

# Output layer (reuse Dense layer)
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_states2 = [state_h2, state_c2]

# Define the inference decoder model
decoder_model = Model(
    [decoder_inputs_single] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)


In [18]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.array([[start_token]])

    # Sampling loop for a batch of sequences
    stop_condition = False
    decoded_sentence = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        decoded_sentence.append(sampled_token_index)

        # Exit condition: either hit max length or find stop token.
        if (sampled_token_index == end_token or len(decoded_sentence) >= maxlen):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.array([[sampled_token_index]])

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [19]:
# Reverse-lookup token index to decode sequences back to words
word_index = imdb.get_word_index()
reverse_word_index = {v+3: k for k, v in word_index.items()}
reverse_word_index[0] = '<PAD>'
reverse_word_index[1] = '<START>'
reverse_word_index[2] = '<UNK>'
reverse_word_index[3] = '<UNUSED>'
reverse_word_index[start_token] = '<START>'
reverse_word_index[end_token] = '<END>'

def sequence_to_text(sequence):
    return ' '.join([reverse_word_index.get(idx, '?') for idx in sequence])


In [20]:
import random

# Select five random samples from the test set
sample_indices = random.sample(range(len(X_test)), 5)
samples = X_test[sample_indices]

for i, sample in enumerate(samples):
    input_seq = sample.reshape(1, -1)
    decoded_sequence = decode_sequence(input_seq)
    print(f"Review {i+1} Original:")
    print(sequence_to_text(sample))
    print(f"\nReview {i+1} Reconstructed:")
    print(sequence_to_text(decoded_sequence))
    print("\n" + "-"*80 + "\n")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━